# Note1_整理與修正 reference 表格
現有的metagenome metatable 本身少了 ecosystem 的欄位資料，需要從該paper的supplemetary table 中的 metagenome table 來進行資料的修正，接著還需要做一張表格是給本次分析，要把terrestrial與aquatic等標籤換至ecosystem中，而不是繼續在ecosystem_category中。

成果:
Reference table 與本次修改的table之間的唯一差別是在 ecosystem的部分，reference中terrestrial 與 aquatic 有著 ecosystem階層: environment，但修改後的版本是把environment給移除，terrestrial與aquatic皆上一一階層。



In [12]:
# link ecosystem information of IMG_metafata to genome_metadata
import pandas as pd


# read files
genome_metadata = pd.read_csv('../data/external/Reference_Data/genome_metadata.csv')
IMG_metadata = pd.read_csv('../data/external/Reference_Data/IMG_metadata.csv')

# extract IMG_TAXON_ID and ECOSYSTEM columns of IMG_metadata
IMG_metadata_IDECO = IMG_metadata[['IMG_TAXON_ID', 'ECOSYSTEM']]

# rename columns
IMG_metadata_IDECO = IMG_metadata_IDECO.rename(columns=({'IMG_TAXON_ID':'metagenome_id'}))

# merge two dataframe on metagenome_id
genome_metadata_ECO = pd.merge(genome_metadata, IMG_metadata_IDECO, on='metagenome_id', how='left')

# rearrange columns
genome_metadata_ECO = genome_metadata_ECO[['genome_id', 'metagenome_id', 'genome_length', 'num_contigs', 'n50',
       'num_16s', 'num_5s', 'num_23s', 'num_trna', 'completeness',
       'contamination', 'quality_score', 'mimag_quality', 'otu_id',
       'ecosystem', 'ECOSYSTEM', 'ecosystem_category', 'ecosystem_type', 'habitat',
       'longitude', 'latitude']]

# rename
genome_metadata_ECO = genome_metadata_ECO.rename(columns=({'ecosystem':'taxonomy', 'ECOSYSTEM':'ecosystem'}))

# save files
genome_metadata_ECO.to_csv('../data/external/Reference_Data/genome_metadata_edit.csv', index=False)

# Next step will perform with excel to change the order about terrestrial and aquatic

# 舊資料

In [3]:
import os
import pandas as pd
import re

Genome_metadata = pd.read_csv('../data/external/Reference_Data/genome_metadata.csv')
print(len(Genome_metadata['metagenome_id'].unique()))

7304


---

date: 2023/02/15

---

## Note1_目標

整理現有相關的資料，包含metagenomes相關的表格資料與HMM分析相關的raw data:

> metagenomes 表格資料
>> - Paper相關資料  
    - Paper: [A genomic catalog of Earth’s microbiomes](https://www.nature.com/articles/s41587-020-0718-6)  
    - Download site: [https://portal.nersc.gov/GEM/](https://portal.nersc.gov/GEM/)  
    - tsv files: ./data/external/Paper_genome_metadata.tsv
>> - 已添加 Biomes data 到 metagenomes 的Table:
    - 目前失敗，有個暫存檔 (BiomesToSub_ecosystem.csv) 在 data/intermin 中
    
> HMM 標準設置資料 (Positive與Negative control)
>> - 執行紀錄: B50_HMM_SettingCriteria_20230203 (Folder)
>> - 相關 HMM output data
    - Actinobacteria  
    Positive control HMM domtblout: 0  
    Negative control HMM domtblout: 0
    
## HMM 標準設置
參考 p-fam 的設置，e-value 為0.001，Database中的序列seaech 設為45638612; -E 0.001 (e-7) -Z 45638612。coverage 設置要大於 50%，可以從domtblout 的結果中用 (ali_to - ali_from + 1) / dtlen) 來計算。最後以negative control來針對各個 hmm profiels設置相應的 bit-score

### Negative 測試



In [40]:
# HMM actino negative control bit-score threshold
import os
import pandas as pd
import re

# A_Rhodococcus_jostii_RHA1
# Define the directory that contains the "domtblout" files, 這個genome protein fasta要把discription的文字刪除，因有空格會影響read
domtblout_dir = "../data/raw/Actino_HMM_Control/Negative/A_Rhodococcus_jostii_RHA1/domtblout/"

# Define the names of the hmmsearches (without the file extension)
hmmsearches = ["A_Rhodococcus_jostii_RHA1_aedC_RS26365", "A_Rhodococcus_jostii_RHA1_aedD_RS26370", "A_Rhodococcus_jostii_RHA1_aedE_RS26375", "A_Rhodococcus_jostii_RHA1_aedI_RS26405", "A_Rhodococcus_jostii_RHA1_aedL_RS26420", "A_Rhodococcus_jostii_RHA1_aedM_RS26425", "A_Rhodococcus_jostii_RHA1_aedN_RS26430", "A_Rhodococcus_jostii_RHA1_aedO_RS26435", "A_Rhodococcus_jostii_RHA1_aedP_RS26440", "A_Rhodococcus_jostii_RHA1_aedQ_RS26445", "A_Rhodococcus_jostii_RHA1_aedR_RS26450", "A_Rhodococcus_jostii_RHA1_I_aedA_RS26385", "A_Rhodococcus_jostii_RHA1_I_aedB_RS26395", "A_Rhodococcus_jostii_RHA1_I_aedF_RS26380", "A_Rhodococcus_jostii_RHA1_I_aedG_RS26390", "A_Rhodococcus_jostii_RHA1_I_aedH_RS26400", "A_Rhodococcus_jostii_RHA1_I_aedJ_RS26410", "A_Rhodococcus_jostii_RHA1_I_aedK_RS26415"]

# Create an empty dictionary to store the best bit-scores for each hmmsearch
best_bit_scores = {}

# Loop over the hmmsearches and parse the corresponding "domtblout" file
for hmmsearch in hmmsearches:
    # Load the "domtblout" file into a pandas DataFrame
    file_path = os.path.join(domtblout_dir, hmmsearch + ".domtblout")
    df = pd.read_csv(file_path, comment="#", sep='\s+', header=None)    
    # Assign column names to the DataFrame
    df.columns = ["target_name", "accession", "tlen", "query_name", "accession2", "qlen", "E-value", "score", "bias",
                  "num_domains_index", "num_domains_total", "c-Evalue", "i-Evalue", "score2", "bias2", "hmm_from", "hmm_to", "ali_from", "ali_to",
                  "env_from", "env_to", "acc", "description"]
    # Calculate the coverage for each hit
    df["coverage"] = (df["ali_to"] - df["ali_from"] + 1) / df["tlen"]

    # Filter the DataFrame by E-value and coverage, and sort by bit-score
    significant_hits = df[(df["E-value"] <= 0.001) & (df["coverage"] > 0.50)].sort_values(by="score", ascending=False)
    
    # replace strains name to aed..
    pattern = r'A_Rhodococcus_jostii_RHA1_'
    hmmsearch = re.sub(pattern, 'A_', hmmsearch)
    
    # Extract the best bit-score and store it in the dictionary
    best_bit_score = significant_hits.iloc[0]["score"]
    best_bit_scores[hmmsearch] = best_bit_score

# Convert the dictionary to a pandas DataFrame and transpose it to get the summary table
summary_table_R = pd.DataFrame(best_bit_scores, index=["best_bit_score_R"])
summary_table_R.insert(0, 'Strains', ['A_Rhodococcus_jostii_RHA1_A'])
summary_table_R






# A_Mycobacterium_tuberculosis_H37Rv
# Define the directory that contains the "domtblout" files, 這個genome protein fasta要把discription的文字刪除，因有空格會影響read
domtblout_dir = "../data/raw/Actino_HMM_Control/Negative/A_Mycobacterium_tuberculosis_H37Rv/domtblout/"

# Define the names of the hmmsearches (without the file extension)
hmmsearches = ['A_Mycobacterium_tuberculosis_H37Rv_aedC_RS26365', 'A_Mycobacterium_tuberculosis_H37Rv_aedD_RS26370', 'A_Mycobacterium_tuberculosis_H37Rv_aedE_RS26375', 'A_Mycobacterium_tuberculosis_H37Rv_aedI_RS26405', 'A_Mycobacterium_tuberculosis_H37Rv_aedL_RS26420', 'A_Mycobacterium_tuberculosis_H37Rv_aedM_RS26425', 'A_Mycobacterium_tuberculosis_H37Rv_aedN_RS26430', 'A_Mycobacterium_tuberculosis_H37Rv_aedO_RS26435', 'A_Mycobacterium_tuberculosis_H37Rv_aedP_RS26440', 'A_Mycobacterium_tuberculosis_H37Rv_aedQ_RS26445', 'A_Mycobacterium_tuberculosis_H37Rv_aedR_RS26450', 'A_Mycobacterium_tuberculosis_H37Rv_I_aedA_RS26385', 'A_Mycobacterium_tuberculosis_H37Rv_I_aedB_RS26395', 'A_Mycobacterium_tuberculosis_H37Rv_I_aedF_RS26380', 'A_Mycobacterium_tuberculosis_H37Rv_I_aedG_RS26390', 'A_Mycobacterium_tuberculosis_H37Rv_I_aedH_RS26400', 'A_Mycobacterium_tuberculosis_H37Rv_I_aedJ_RS26410', 'A_Mycobacterium_tuberculosis_H37Rv_I_aedK_RS26415']
# Create an empty dictionary to store the best bit-scores for each hmmsearch
best_bit_scores = {}

# Loop over the hmmsearches and parse the corresponding "domtblout" file
for hmmsearch in hmmsearches:
    # Load the "domtblout" file into a pandas DataFrame
    file_path = os.path.join(domtblout_dir, hmmsearch + ".domtblout")
    df = pd.read_csv(file_path, comment="#", sep='\s+', header=None)    
    # Assign column names to the DataFrame
    df.columns = ["target_name", "accession", "tlen", "query_name", "accession2", "qlen", "E-value", "score", "bias",
                  "num_domains_index", "num_domains_total", "c-Evalue", "i-Evalue", "score2", "bias2", "hmm_from", "hmm_to", "ali_from", "ali_to",
                  "env_from", "env_to", "acc", "description"]
    # Calculate the coverage for each hit
    df["coverage"] = (df["ali_to"] - df["ali_from"] + 1) / df["tlen"]

    # Filter the DataFrame by E-value and coverage, and sort by bit-score
    significant_hits_1 = df[(df["E-value"] <= 0.001) & (df["coverage"] > 0.50)].sort_values(by="score", ascending=False)
   
    # replace strains name to aed..
    pattern = r'A_Mycobacterium_tuberculosis_H37Rv_'
    hmmsearch = re.sub(pattern, 'A_', hmmsearch)

    # Extract the best bit-score and store it in the dictionary
    if not significant_hits_1.empty:
        best_bit_score = significant_hits_1.iloc[0]["score"]
        best_bit_scores[hmmsearch] = best_bit_score
    else:
        best_bit_score = 0
        best_bit_scores[hmmsearch] = best_bit_score

# Convert the dictionary to a pandas DataFrame and transpose it to get the summary table
summary_table_M = pd.DataFrame(best_bit_scores, index=["best_bit_score_M"])
summary_table_M.insert(0, 'Strains', ['Mycobacterium_tuberculosis_H37Rv'])
summary_table_M

# merge two negative best bit score df
Actino_Negative_df = pd.concat([summary_table_R, summary_table_M], axis=0)
Actino_Negative_df

# Got the smallest bit score of the df
Actino_BitScore_Criteria = Actino_Negative_df.max()
Actino_BitScore_Criteria = Actino_BitScore_Criteria.iloc[1:]

# Change index name for MAGs data parse
for old_index in Actino_BitScore_Criteria.index:
    Actino_BitScore_Criteria.rename(index={old_index: re.sub('A_aed', 'MAGs_A_aed', old_index)}, inplace=True)

for old_index in Actino_BitScore_Criteria.index:
    Actino_BitScore_Criteria.rename(index={old_index: re.sub('A_I_aed', 'MAGs_A_I_aed', old_index)}, inplace=True)
    
Actino_BitScore_Criteria = Actino_BitScore_Criteria.astype(float)
    
Actino_BitScore_Criteria


MAGs_A_aedC_RS26365      450.3
MAGs_A_aedD_RS26370      128.9
MAGs_A_aedE_RS26375      232.1
MAGs_A_aedI_RS26405       25.4
MAGs_A_aedL_RS26420      106.7
MAGs_A_aedM_RS26425       98.3
MAGs_A_aedN_RS26430      221.1
MAGs_A_aedO_RS26435       58.9
MAGs_A_aedP_RS26440      181.3
MAGs_A_aedQ_RS26445       42.4
MAGs_A_aedR_RS26450      255.5
MAGs_A_I_aedA_RS26385    195.4
MAGs_A_I_aedB_RS26395    220.3
MAGs_A_I_aedF_RS26380    617.3
MAGs_A_I_aedG_RS26390    176.2
MAGs_A_I_aedH_RS26400    270.8
MAGs_A_I_aedJ_RS26410    328.3
MAGs_A_I_aedK_RS26415    321.5
dtype: float64

In [72]:
# Actino_HMM_MAGs

import os
import pandas as pd
import re

# Aed Cluster to MAGs
# Define the directory that contains the "domtblout" files.需要刪除discription
domtblout_dir = "../data/raw/Actino_HMM_MAGs_domtblout/"

# Create an empty dictionary to store the target name for each hmmsearch
MAGs_Hits = {}
MAGs_Hits_name = []

# delet aedC,Q,R. They are not in next hmm analysis
to_drop = ['MAGs_A_aedC_RS26365', 'MAGs_A_aedQ_RS26445', 'MAGs_A_aedR_RS26450']
Actino_BitScore_Criteria = Actino_BitScore_Criteria.loc[~Actino_BitScore_Criteria.index.isin(to_drop)]

# create a dataframe for a all hits 
columns = ["target_name", "accession", "tlen", "query_name", "accession2", "qlen", "E-value", "score", "bias",
           "num_domains_index", "num_domains_total", "c-Evalue", "i-Evalue", "score2", "bias2", "hmm_from",
           "hmm_to", "ali_from", "ali_to", "env_from", "env_to", "acc", "description"]

All_aed_Hits_df = pd.DataFrame(columns=columns)

# Loop over the HMM DOMTBLOUT files and filter the results based on bit score, e-value and coverage
# hmm name and bit score are in the Actino_BitScore_Criteria series
for hmmsearch, threshold in Actino_BitScore_Criteria.items():
    # Load the "domtblout" file into a pandas DataFrame
    file_path = os.path.join(domtblout_dir, hmmsearch + ".domtblout")
    df = pd.read_csv(file_path, comment="#", sep='\s+', header=None)    
    # Assign column names to the DataFrame
    df.columns = ["target_name", "accession", "tlen", "query_name", "accession2", "qlen", "E-value", "score", "bias",
                  "num_domains_index", "num_domains_total", "c-Evalue", "i-Evalue", "score2", "bias2", "hmm_from", "hmm_to", "ali_from", "ali_to",
                  "env_from", "env_to", "acc", "description"]

    # Calculate the coverage for each hit
    df["coverage"] = (df["ali_to"] - df["ali_from"] + 1) / df["tlen"]
    
    # Filter the DataFrame by E-value, coverage, and bit-score
    significant_hits = df[(df["E-value"] <= 0.001) & (df["coverage"] > 0.50) & (df["score"] > threshold)]

    # Extract Target nmae and store it in the dictionary
    if not significant_hits.empty:
        MAGs_Hits_name = significant_hits["target_name"].tolist()
        MAGs_Hits[hmmsearch] = MAGs_Hits_name
    else:
        MAGs_Hits_name = None
        MAGs_Hits[hmmsearch] = MAGs_Hits_name
    
    # add hits table to a df
    All_aed_Hits_df = pd.concat([significant_hits, All_aed_Hits_df], axis=0)

# All_aed_Hits_df.to_csv('../data/processed/All_aed_Hits_df.csv')    
    
All_aed_Hits_TargetAndQuery = All_aed_Hits_df[['target_name', 'query_name']]




In [ ]:
# use python to get the MAGs ID of Target gene

# import os
# import re
# import time
# import glob
# # aed_All target gene

# time_start = time.time()
# count = 0
# aed_All_HitGenome = open("..\data\interim\actino_HMMHitGeneToMAGsID.txt", 'w+', encoding = 'utf-8') #建立檔案


In [92]:
# link target name to metagenome dataframe

# Load a Dataframe with the lookup values for merge protein id to MAGs and merge them
TarToMAGs_aed = pd.read_csv('../data/interim/aed_TargetToMAGsID_all.csv')

# use merge() function to join the MAGsID data
aed_hits_TargetAndMAGsID = pd.merge(All_aed_Hits_TargetAndQuery, TarToMAGs_aed, on='target_name', how='left')                                                                               
aed_hits_TargetAndMAGsID

# # remove duplicates in the 'MAGsID' column
# a = aed_hits_TargetAndMAGsID['query_name'].drop_duplicates()
# a

# # Find the rows with at least one NaN value
# nan_rows = aed_hits_TargetAndMAGsID.isna().any(axis=1)
# aed_hits_nan_rows = aed_hits_TargetAndMAGsID[nan_rows]
# aed_hits_nan_rows.to_csv('../data/interim/aed_hits_nan_rows.csv')

# create the crosstab table (like heatmap)
aed_hits_heatmap = pd.crosstab(aed_hits_TargetAndMAGsID['query_name'], aed_hits_TargetAndMAGsID['MAGsID'], dropna=False)
aed_hits_heatmap = aed_hits_heatmap.transpose()
aed_hits_heatmap.to_csv('../data/processed/aed_hits_heatmap_bitscore.csv')






## 下列為暫不執行的cell

In [1]:
import os
import pandas as pd
import re

TarToMAGs_null = pd.read_csv('../data/interim/null_TarToMAGs.csv')
TarToMAGs_aed_All = pd.read_csv('../data/interim/Actino_aed/aed_All_TarToMAGsID.csv')
TarToMAGs_aed_All = pd.concat([TarToMAGs_aed_All, TarToMAGs_null], axis=0)
TarToMAGs_aed_All.to_csv('../data/interim/Actino_aed/aed_All_TarToMAGsID.csv')
print('done')
# TarToMAGs_aedCB = pd.read_csv('../data/interim/aedC_aedB.csv')
# TarToMAGs_aedHR = pd.read_csv('../data/interim/aedH_aedR.csv')
# TarToMAGs_aed_Non = pd.read_csv('../data/interim/aed_Non_TarToMAGsID.csv')
# TarToMAGs_aed_All = pd.concat([TarToMAGs_aedCB, TarToMAGs_aedHR], axis=0)
# TarToMAGs_aed_All = pd.concat([TarToMAGs_aed_All, TarToMAGs_aed_Non], axis=0)
# print(TarToMAGs_aed_All.shape[0])
# TarToMAGs_aed_All
# TarToMAGs_aed_All = TarToMAGs_aed_All.drop_duplicates(subset='target_name')
# print(TarToMAGs_aed_All.shape[0])
# TarToMAGs_aed_All.to_csv('../data/interim/aed_All_TarToMAGsID.csv')
# # TarToMAGs_aed_All.to_csv('../data/interim/aed_TargetToMAGsID_all.csv')




# # All_aed_Hits_TargetAndQuery


# # for test
# import os
# import pandas as pd
# import re

# # Aed Cluster to MAGs
# # Define the directory that contains the "domtblout" files.需要刪除discription
# domtblout_dir = "../data/raw/Actino_HMM_MAGs_domtblout/"

# # Create an empty dictionary to store the target name for each hmmsearch
# MAGs_Hits = {}
# MAGs_Hits_name = []

# # delet aedC,Q,R. They are not in next hmm analysis
# to_drop = ['MAGs_A_aedC_RS26365', 'MAGs_A_aedQ_RS26445', 'MAGs_A_aedR_RS26450']
# Actino_BitScore_Criteria = Actino_BitScore_Criteria.loc[~Actino_BitScore_Criteria.index.isin(to_drop)]

# # create a dataframe for a all hits 
# columns = ["target_name", "accession", "tlen", "query_name", "accession2", "qlen", "E-value", "score", "bias",
#            "num_domains_index", "num_domains_total", "c-Evalue", "i-Evalue", "score2", "bias2", "hmm_from",
#            "hmm_to", "ali_from", "ali_to", "env_from", "env_to", "acc", "description"]

# All_Hits_df = pd.DataFrame(columns=columns)

# # Loop over the HMM DOMTBLOUT files and filter the results based on bit score, e-value and coverage
# # hmm name and bit score are in the Actino_BitScore_Criteria series
# for hmmsearch, threshold in Actino_BitScore_Criteria.items():
#     # Load the "domtblout" file into a pandas DataFrame
#     file_path = os.path.join(domtblout_dir, hmmsearch + ".domtblout")
#     df = pd.read_csv(file_path, comment="#", sep='\s+', header=None)    
#     # Assign column names to the DataFrame
#     df.columns = ["target_name", "accession", "tlen", "query_name", "accession2", "qlen", "E-value", "score", "bias",
#                   "num_domains_index", "num_domains_total", "c-Evalue", "i-Evalue", "score2", "bias2", "hmm_from", "hmm_to", "ali_from", "ali_to",
#                   "env_from", "env_to", "acc", "description"]

#     # Calculate the coverage for each hit
#     df["coverage"] = (df["ali_to"] - df["ali_from"] + 1) / df["tlen"]
    
#     # Filter the DataFrame by E-value, coverage, and bit-score
#     significant_hits = df[(df["E-value"] <= 0.001) & (df["coverage"] > 0.50) & (df["score"] > threshold)]

#     # Extract Target nmae and store it in the dictionary
#     if not significant_hits.empty:
#         MAGs_Hits_name = significant_hits["target_name"].tolist()
#         MAGs_Hits[hmmsearch] = MAGs_Hits_name
#     else:
#         MAGs_Hits_name = None
#         MAGs_Hits[hmmsearch] = MAGs_Hits_name

# a = 0        
# for key in MAGs_Hits:
#     a += len(MAGs_Hits[key])
    
# # a 25082

done


In [ ]:
# # Install the pandas package using pip
# !pip install pandas

In [62]:
# # The goal is to add the Biomes data into the metagenome table.
# # pause the shell. It fails

# # Import the pandas package
# import pandas as pd

# # Load the TSV data into a DataFrame
# MetaDf = pd.read_csv('..\data\external\Paper_genome_metadata.tsv', delimiter='\t', header=0)
# # -print(MetaDf.head())

# # Load a Dataframe with the lookup values
# BioToEco = pd.read_csv('..\data\interim\BiomesToSub_ecosystem.csv')
# # -print(BioToEco.head())

# # To prepare for merging Biomes data with MetaDf, extract the relevant column to be used. Keys are ecosystem data.

# Bio_Eco1 = BioToEco.loc[:,['Biomes', 'ecosystem_category']]

# Bio_Eco2 = BioToEco.loc[:,['Biomes', 'ecosystem_type']]

# Bio_Eco3 = BioToEco.loc[:,['Biomes', 'habitat']]

# # use merge() function to join the Biomes data

# MetaDf_Bio_Category = pd.merge(MetaDf, Bio_Eco1, on='ecosystem_category', how='left')

# MetaDf_Bio_type = pd.merge(MetaDf_Bio_Category, Bio_Eco2, on='ecosystem_type', how='left')

# MetaDf_Bio_habitat = pd.merge(MetaDf_Bio_type, Bio_Eco3, on='habitat', how='right')
# print(MetaDf_Bio_habitat)
# # MetaDf_Bio = pd.merge(pd.merge(pd.merge(MetaDf_Bio_Ecosystem, MetaDf_Bio_Category, on='genome_id'), MetaDf_Bio_type, on='genome_id'), MetaDf_Bio_habitat, on='genome_id')
# # MetaDf_Bio

# # # MetaDf_Bio_columns = MetaDf_Bio.columns

# # # # Change the column order
# # # -MetaDf_Bio_columns
# # # MetaDf_Bio = MetaDf_Bio[['genome_id', 'metagenome_id', 'genome_length', 'num_contigs', 'n50',
# # #        'num_16s', 'num_5s', 'num_23s', 'num_trna', 'completeness',
# # #        'contamination', 'quality_score', 'mimag_quality', 'otu_id',
# # #        'ecosystem', 'Biomes', 'ecosystem_category', 'ecosystem_type', 'habitat',
# # #        'longitude', 'latitude']]
# # # # -MetaDf_Bio

# # # MetaDf_Bio.to_csv('..\data\processed\Table\MetagenomeWithBiomes.csv', index=False)
# # # # 確認過後MetaDf_Bio還須修改，因為要分開merge，接續merge的話資料只會呈現在ecosystem有隊到 key 的資料